In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pyspark as ps

spark = ps.sql.SparkSession.builder \
        .master("local[64]") \
        .appName("df lecture") \
        .getOrCreate()

sc = spark.sparkContext

In [118]:
# %load_ext autoreload
%autoreload 2
from pyspark.sql import SQLContext
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.metrics import confusion_matrix
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, HashingTF, IDF, CountVectorizer
from pyspark.ml.classification import LogisticRegression as spark_LogisticRegression
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
import cleaner

%matplotlib inline

In [17]:
f_path = "s3://chizzy/yelp_data/"
df_bus = pd.read_json(f_path+"business.json", lines=True)
df_user = pd.read_json(f_path+"user.json", lines=True)
df_rev = pd.read_json(f_path+"review.json", lines=True)

In [18]:
df_bus.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,"4855 E Warner Rd, Ste B9","{'AcceptsInsurance': True, 'ByAppointmentOnly'...",FYWN1wneV18bWNgQjJ2GNg,"[Dentists, General Dentistry, Health & Medical...",Ahwatukee,"{'Friday': '7:30-17:00', 'Tuesday': '7:30-17:0...",1,33.330690,-111.978599,Dental by Design,,85044,22,4.0,AZ
1,3101 Washington Rd,"{'BusinessParking': {'garage': False, 'street'...",He-G7vWjzVUysIKrfNbPUQ,"[Hair Stylists, Hair Salons, Men's Hair Salons...",McMurray,"{'Monday': '9:00-20:00', 'Tuesday': '9:00-20:0...",1,40.291685,-80.104900,Stephen Szabo Salon,,15317,11,3.0,PA
2,"6025 N 27th Ave, Ste 1",{},KQPW8lFf1y5BT2MxiSZ3QA,"[Departments of Motor Vehicles, Public Service...",Phoenix,{},1,33.524903,-112.115310,Western Motor Vehicle,,85017,18,1.5,AZ
3,"5000 Arizona Mills Cr, Ste 435","{'BusinessAcceptsCreditCards': True, 'Restaura...",8DShNS-LuFqpEWIp0HxijA,"[Sporting Goods, Shopping]",Tempe,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",0,33.383147,-111.964725,Sports Authority,,85282,9,3.0,AZ
4,581 Howe Ave,"{'Alcohol': 'full_bar', 'HasTV': True, 'NoiseL...",PfOCPjBrlQAnz__NXj9h_w,"[American (New), Nightlife, Bars, Sandwiches, ...",Cuyahoga Falls,"{'Monday': '11:00-1:00', 'Tuesday': '11:00-1:0...",1,41.119535,-81.475690,Brick House Tavern + Tap,,44221,116,3.5,OH


In [19]:
df_user.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,4.67,0,0,0,0,0,0,0,0,1,...,0,[],0,"[cvVMmlU1ouS3I5fhutaryQ, nj6UZ8tdGo8YJ9lUMTVWN...",0,Johnny,8,0,oMy_rEb0UBEmMlu-zcxnoQ,2014-11-03
1,3.70,0,0,0,0,0,0,0,0,0,...,0,[],0,"[0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gp...",0,Chris,10,0,JJ-aSuM4pCFPdkfoZ34q0Q,2013-09-24
2,2.00,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Tiffy,1,0,uUzsFQn_6cXDh6rPNGbIFA,2017-03-02
3,4.67,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Mark,6,0,mBneaEEH5EMyxaVyqS-72A,2015-03-13
4,4.67,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Evelyn,3,0,W5mJGs-dcDWRGEhAzUYtoA,2016-09-08


In [20]:
df_rev.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw


In [66]:
#Rename colume names 
df_bus.rename(columns={"name":"bus_name","review_count":"bus_rev_count"}, inplace=True)
df_user.rename(columns={"name":"user_name", "review_count":"user_rev_count"}, inplace=True)
df_rev.rename(columns={"cool":"cool_rev", "funny":"funny_rev", "stars":"stars_rev", "useful":"useful_rev"}, inplace=True)

In [67]:
print("Number of rows for business table: {}".format(len(df_bus)))
print("Number of rows for user table: {}".format(len(df_user)))
print("Number of rows for review table: {}".format(len(df_rev)))

Number of rows for business table: 128302
Number of rows for user table: 1326101
Number of rows for review table: 5261669


In [68]:
us_states = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS", \
            "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY", \
            "NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]

#Extracting reviews for only US restaurants
df_bus_us = df_bus[df_bus.state.isin (us_states)]

In [69]:
print("Number of rows for business table for US states: {}".format(len(df_bus_us)))

Number of rows for business table for US states: 128302


In [70]:
df_rev_bus = pd.merge(df_rev, df_bus, how='inner', on=['business_id', 'business_id'])

In [71]:
df_rev_bus.head()

,business_id,cool_rev,date,funny_rev,review_id,stars_rev,text,useful_rev,user_id,address,...,hours,is_open,latitude,longitude,bus_name,neighborhood,postal_code,bus_rev_count,stars,state
0,8QWPlVQ6D-OExqXoaD2Z1g,0,2014-09-24,0,HRPm3vEZ_F-33TYVT7Pebw,5,Cycle Pub Las Vegas was a blast! Got a groupon...,1,_4iMDXbXZ1p1ONG297YEAQ,201 N 3rd St,...,{},1,36.171869,-115.142146,Vegas Pub Crawler,Downtown,89101,38,4.5,NV
1,8QWPlVQ6D-OExqXoaD2Z1g,3,2014-03-12,2,31uIbU4c10W5X5Fv506ryA,4,Cycle pub it is......\nTalk about having a gre...,6,JlwWHBFT76iSJe5mWIcZ4A,201 N 3rd St,...,{},1,36.171869,-115.142146,Vegas Pub Crawler,Downtown,89101,38,4.5,NV
2,8QWPlVQ6D-OExqXoaD2Z1g,0,2014-04-15,0,3WBCOXjwKnuu01pNljDKGg,5,I've wanted to do Cycle Pub since spying it at...,6,Q6CEHR-6P-gQbNtzCAj4Ig,201 N 3rd St,...,{},1,36.171869,-115.142146,Vegas Pub Crawler,Downtown,89101,38,4.5,NV
3,8QWPlVQ6D-OExqXoaD2Z1g,0,2017-08-04,0,3AbYdLNW2NbGsD_YM1a78w,5,Had such a great time. Now I want to do Vegas ...,0,WF0_ES4qpeBqBxqqTLyT4g,201 N 3rd St,...,{},1,36.171869,-115.142146,Vegas Pub Crawler,Downtown,89101,38,4.5,NV
4,8QWPlVQ6D-OExqXoaD2Z1g,0,2016-03-30,0,XIzPfkbRNMumLLk5qYndDQ,5,"Ben is awesome! Fun, fantastic, wonderful expe...",0,VPA1wfF_-_bwNmyTgELRnw,201 N 3rd St,...,{},1,36.171869,-115.142146,Vegas Pub Crawler,Downtown,89101,38,4.5,NV


In [72]:
df_rev_bus_user = pd.merge(df_rev_bus, df_user, how='left', on=['user_id', 'user_id'])

In [73]:
print("Number of rows for business table for concated business, user and review: {}".format(len(df_rev_bus_user)))

Number of rows for business table for concated business, user and review: 4391457


In [74]:
df_rev_bus_user.head()

,business_id,cool_rev,date,funny_rev,review_id,stars_rev,text,useful_rev,user_id,address,...,compliment_writer,cool,elite,fans,friends,funny,user_name,user_rev_count,useful,yelping_since
0,8QWPlVQ6D-OExqXoaD2Z1g,0,2014-09-24,0,HRPm3vEZ_F-33TYVT7Pebw,5,Cycle Pub Las Vegas was a blast! Got a groupon...,1,_4iMDXbXZ1p1ONG297YEAQ,201 N 3rd St,...,0,0,[],0,[],0,Robert,1,0,2012-05-13
1,8QWPlVQ6D-OExqXoaD2Z1g,3,2014-03-12,2,31uIbU4c10W5X5Fv506ryA,4,Cycle pub it is......\nTalk about having a gre...,6,JlwWHBFT76iSJe5mWIcZ4A,201 N 3rd St,...,57,1636,"[2015, 2014, 2016]",23,"[E19IMURE4WEoCnvDx-bylg, kJ9Wd9nq5BV0bSA9V98mV...",1091,Trev,212,1392,2014-02-06
2,8QWPlVQ6D-OExqXoaD2Z1g,0,2014-04-15,0,3WBCOXjwKnuu01pNljDKGg,5,I've wanted to do Cycle Pub since spying it at...,6,Q6CEHR-6P-gQbNtzCAj4Ig,201 N 3rd St,...,0,7,[],0,"[kiao9cLyffXHVth0B1FwfA, lcf0ZgVioqgFzU5QW73am...",0,Jaime,2,0,2013-04-27
3,8QWPlVQ6D-OExqXoaD2Z1g,0,2017-08-04,0,3AbYdLNW2NbGsD_YM1a78w,5,Had such a great time. Now I want to do Vegas ...,0,WF0_ES4qpeBqBxqqTLyT4g,201 N 3rd St,...,0,0,[],1,[],2,Lexy,17,2,2010-07-10
4,8QWPlVQ6D-OExqXoaD2Z1g,0,2016-03-30,0,XIzPfkbRNMumLLk5qYndDQ,5,"Ben is awesome! Fun, fantastic, wonderful expe...",0,VPA1wfF_-_bwNmyTgELRnw,201 N 3rd St,...,0,0,[],0,[],0,Cee,2,0,2014-12-06


In [75]:
df_rev_bus_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4391457 entries, 0 to 4391456
Data columns (total 44 columns):
business_id           object
cool_rev              int64
date                  datetime64[ns]
funny_rev             int64
review_id             object
stars_rev             int64
text                  object
useful_rev            int64
user_id               object
address               object
attributes            object
categories            object
city                  object
hours                 object
is_open               int64
latitude              float64
longitude             float64
bus_name              object
neighborhood          object
postal_code           object
bus_rev_count         int64
stars                 float64
state                 object
average_stars         float64
compliment_cool       int64
compliment_cute       int64
compliment_funny      int64
compliment_hot        int64
compliment_list       int64
compliment_more       int64
compliment_note  

In [4]:
# review_df = spark.read.json('../break_week/data/dataset/review.json')
# user_df = spark.read.json('../break_week/data/dataset/user.json')
# business_df = spark.read.json("../break_week/data/dataset/business.json")

In [5]:
# review_df.createTempView("review")
# user_df.createTempView("user")
# business_df.createTempView("business")

In [6]:
# df = spark.sql("""SELECT new.user_name, new.user_id, new.business_id, new.friends, \
#                 b.name AS business_name, b.state, b.city, b.address, b.categories, b.stars AS bus_star,\
#                 new.text, new.stars AS review_star \
#                 FROM \
#                     (SELECT u.name AS user_name, r.user_id, r.business_id, r.text, r.stars, u.friends \
#                     FROM review AS r \
#                     LEFT JOIN user AS u \
#                     ON r.user_id = u.user_id) AS new\
#                 INNER JOIN business as b\
#                 ON new.business_id = b.business_id \
#                 WHERE ARRAY_CONTAINS(b.categories, 'Restaurants') \
#                 AND b.state IN ("AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS", \
#                                 "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY", \
#                                 "NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY") """)

In [57]:
# df.count()

In [58]:
# df_1_5 = df.filter("review_star = 1 OR review_star = 5")
# df_1_5.show(2)

In [60]:
# df_1_5.count()

In [61]:
# df_1_5.select("state").groupBy("state").count().show(50,False)

In [62]:
# df1 = df_1_5.where("state = 'WI'").select(["text", "review_star"])

In [63]:
# tokenizer = Tokenizer(inputCol="text", outputCol="words")
# wordsData = tokenizer.transform(df1)

In [12]:
# df_wi = df1.toPandas()

In [77]:
df = df_rev_bus_user[["text", "stars_rev"]]
df.head()

,text,stars_rev
0,Cycle Pub Las Vegas was a blast! Got a groupon...,5
1,Cycle pub it is......\nTalk about having a gre...,4
2,I've wanted to do Cycle Pub since spying it at...,5
3,Had such a great time. Now I want to do Vegas ...,5
4,"Ben is awesome! Fun, fantastic, wonderful expe...",5


In [99]:
df1 = df.head(80000)
corpus = df1["text"]

In [100]:
corpus[3]

"Had such a great time. Now I want to do Vegas Pub Crawler every time I come to Vegas. There were three separate groups on our ride. Two larger groups and then me & my husband. We used a Groupon. All you need to do is show it to the driver on your phone/printed out and he'll take a photo of it. Super easy. We were all a bit timid at first, but our driver Jon loosened us up and kept us happy! We spent about 20 minutes at 5 spots. They were all decently close - a few blocks apart. There is no drinking on the actual vehicle. But that wasn't an issue since we were stopping and going quickly. You don't actually have to pedal at all. It's totally motorized. Also, we went at 9p which was a great way to avoid the Vegas sun. Such a memorable way to spend time in Vegas and check out bars you didn't know about!"

In [101]:
print("Number of words befor cleaning, and removing stopwords: {}".format(len(corpus[3].split())))

Number of words befor cleaning, and removing stopwords: 164


In [102]:
# nltk.download('stopwords')
# nltk.download('wordnet')

In [103]:
sw = set(stopwords.words("english"))
tokenizer = RegexpTokenizer("[\w']+")
st = PorterStemmer()
lemma = WordNetLemmatizer()

In [104]:
cleaned = cleaner.clean_stem(corpus, tokenizer, lemma, sw)

In [105]:
cleaned[3]

"had great time now i want vega pub crawler every time i come vega there three separate group ride two larger group husband we used groupon all need show driver phone printed he'll take photo super easy we bit timid first driver jon loosened u kept u happy we spent minute spot they decently close block apart there drinking actual vehicle but issue since stopping going quickly you actually pedal it's totally motorized also went 9p great way avoid vega sun such memorable way spend time vega check bar know"

In [106]:
tfidf_vectorizer= TfidfVectorizer(max_df=0.95, min_df=2,
                                stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(cleaned)

In [98]:
X = tfidf.toarray()
X.shape

(80000, 28562)

In [107]:
y = df1["stars_rev"].values

In [132]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [133]:
# nb = GaussianNB()
# nb.fit(X_train, y_train)

### LogisticRegression Linear model

In [134]:
lreg = LogisticRegression()
lreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [135]:
y_pred = lreg.predict(X_test)

In [136]:
def confu(y_test, y_pred):
    tp, tn, fp, fn = 0,0,0,0
    for x,y in zip(y_test, y_pred):
        if x == y:
            if x > 3:
                tp+=1
            else:
                tn+=1
        elif x != y:
            if y>3:
                fp+=1
            else:
                fn+1
    return tp, tn, fp, fn

In [137]:
def metrics(y_test, y_pred):
    tp, tn, fp, fn = confu(y_test, y_pred)
    matrix = np.array([[tp, fp], [fn, tn]])
    recall = tp/(tp+fn)
    precision = tp/(fp+tp)
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    return matrix, recall, precision, accuracy

In [129]:
matrix, recall, precision, accuracy = metrics(y_test, y_pred)

In [138]:
#Estimating evaluation metrics
# matrix, recall, precision, accuracy = cleaner.metrics(y_test=y_test, y_predict=y_pred)

In [139]:
#confusion matrix
matrix

array([[8699, 5799],
       [   0, 3092]])

In [140]:
print("Recall: {}%".format(round(recall*100, 2)))
print("Precision: {}%".format(round(precision*100, 2)))
print("Accuracy: {}%".format(round(accuracy*100, 2)))

Recall: 100.0%
Precision: 60.0%
Accuracy: 67.03%


In [141]:
terms = tfidf_vectorizer.get_feature_names()

In [144]:
# df_1_5.select("review_star").groupBy("review_star").count().show()

In [145]:
pos_terms, neg_terms = cleaner.show_topics(lreg.coef_, terms, length=40)

In [146]:
pos_terms

[('best', 6.80288704363611),
 ('amazing', 6.622382200229562),
 ('awesome', 4.535405439351903),
 ('delicious', 4.277399571478582),
 ('perfect', 4.065658104108061),
 ('thank', 3.994333611317463),
 ('love', 3.968631816344425),
 ('excellent', 3.9313464274445726),
 ('favorite', 3.926740625353595),
 ('highly', 3.9040771553051092),
 ('fantastic', 3.728577419700471),
 ('incredible', 3.6323654294201813),
 ('outstanding', 3.492029098096073),
 ('great', 3.360544985924977),
 ('wonderful', 3.3207934317339034),
 ('omg', 3.193340716373476),
 ('bomb', 3.1786127401807396),
 ('heaven', 3.0898723003510087),
 ('perfectly', 3.084994302889382),
 ('perfection', 3.0413323922309328),
 ('professional', 3.032409150813298),
 ('gem', 2.96818216419987),
 ('loved', 2.839264735031425),
 ('notch', 2.8185192761240714),
 ('phenomenal', 2.7314482232615167),
 ('deserves', 2.6504540536066648),
 ('die', 2.6448899743417615),
 ('holy', 2.5746937359281943),
 ('ryan', 2.565273268148959),
 ('wow', 2.5320142204333966),
 ('excepti

In [147]:
neg_terms

[('worst', -6.270885271731235),
 ('ok', -5.932618592623465),
 ('bland', -5.784663404694461),
 ('okay', -5.457155411756951),
 ('rude', -5.393969482220618),
 ('horrible', -5.3169206990282305),
 ('terrible', -5.161410966603913),
 ('mediocre', -4.944914024164714),
 ('average', -4.572687372075172),
 ('disappointing', -4.441483613712197),
 ('slow', -4.403267251308377),
 ('overpriced', -4.278535670122588),
 ('pretty', -4.187652856054678),
 ('meh', -3.768321124043735),
 ('decent', -3.6601788765431396),
 ('overall', -3.5415174070826936),
 ('dirty', -3.4601718107756985),
 ('poor', -3.4241857182983173),
 ('reason', -3.403739760443927),
 ('star', -3.3394465144306626),
 ('dry', -3.2499340101487575),
 ('awful', -3.2158835748545647),
 ('guess', -3.148097172897898),
 ('solid', -3.1275940302188325),
 ('lack', -3.123415473134199),
 ('unfortunately', -3.1074134110982423),
 ('alright', -3.0996945433182184),
 ('asked', -3.0867740580455947),
 ('cold', -3.064045415802472),
 ('disgusting', -3.0448264900847817

### SPARK DATAFRAME PIPELINE

In [618]:
df_spk = df1.limit(7000)
df_spk.show(2)

+--------------------+-----------+
|                text|review_star|
+--------------------+-----------+
|Enjoyed a delicio...|          5|
|Had a great time ...|          5|
+--------------------+-----------+
only showing top 2 rows



In [628]:
df_test, df_train = df_spk.randomSplit(weights=[0.3, 0.7], seed=1)

In [629]:
print("Number of rows for trainset: {}".format(df_train.count()))
print("Number of rows for testset: {}".format(df_test.count()))

Number of rows for trainset: 4876
Number of rows for testset: 2124


In [630]:
regex_tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
tokenized = regex_tokenizer.transform(df_train)

In [631]:
countTokens = udf(lambda words: len(words), IntegerType())

In [632]:
tokenized_df = tokenized.select("text", "words", "review_star") \
    .withColumn("tokens", countTokens(col("words")))

In [633]:
stops_removed_df = remover.transform(tokenized_df)
stops_removed_df.show(3, truncate=True)

+--------------------+--------------------+-----------+------+--------------------+
|                text|               words|review_star|tokens|            filtered|
+--------------------+--------------------+-----------+------+--------------------+
|"A once in a life...|[a, once, in, a, ...|          5|   180|[lifetime, experi...|
|"Forrest, what's ...|[forrest, what, s...|          5|   304|[forrest, going, ...|
|"Garbage baked in...|[garbage, baked, ...|          1|   210|[garbage, baked, ...|
+--------------------+--------------------+-----------+------+--------------------+
only showing top 3 rows



In [634]:
input_df = stops_removed_df.selectExpr("filtered", "review_star as label")
input_df.show(1)

+--------------------+-----+
|            filtered|label|
+--------------------+-----+
|[lifetime, experi...|    5|
+--------------------+-----+
only showing top 1 row



In [337]:
input_df.select("filtered").show(4, False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|filtered                                                                                                                                                                                                                                                                                                                                                                       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [635]:
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures")
featurizedData = hashingTF.transform(input_df)

In [640]:
featurizedData.show(1)

+--------------------+-----+--------------------+
|            filtered|label|         rawFeatures|
+--------------------+-----+--------------------+
|[lifetime, experi...|    5|(262144,[8443,880...|
+--------------------+-----+--------------------+
only showing top 1 row



In [641]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
tfidfModel = idf.fit(featurizedData).transform(featurizedData)

Py4JJavaError: An error occurred while calling o5957.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 956.0 failed 1 times, most recent failure: Lost task 0.0 in stage 956.0 (TID 24925, localhost, executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1517)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1505)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1504)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1504)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:814)
	at or

In [495]:
tfidfModel.show(2, False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [496]:
tfidfModel.select("features").show(1, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
lr = LogisticRegression(maxIter=10, regParam=0.01)
model = lr.fit(tfidfModel)

In [498]:
#testing the model

In [499]:
# df_test = df1.subtract(df_train).limit(1250)
# df_test.show(1)

+--------------------+-----------+
|                text|review_star|
+--------------------+-----------+
|I was visiting Ma...|          5|
+--------------------+-----------+
only showing top 1 row



### Building test TFIDF matrix

In [637]:
tokenized_test = regex_tokenizer.transform(df_test)


In [638]:
countTokens = udf(lambda words: len(words), IntegerType())

In [632]:
tokenized_df = tokenized_test.select("text", "words", "review_star") \
    .withColumn("tokens", countTokens(col("words")))

In [633]:
stops_removed_df = remover.transform(tokenized_df)
stops_removed_df.show(3, truncate=True)

+--------------------+--------------------+-----------+------+--------------------+
|                text|               words|review_star|tokens|            filtered|
+--------------------+--------------------+-----------+------+--------------------+
|"A once in a life...|[a, once, in, a, ...|          5|   180|[lifetime, experi...|
|"Forrest, what's ...|[forrest, what, s...|          5|   304|[forrest, going, ...|
|"Garbage baked in...|[garbage, baked, ...|          1|   210|[garbage, baked, ...|
+--------------------+--------------------+-----------+------+--------------------+
only showing top 3 rows



In [500]:
input_df_test = stops_removed_df_test.selectExpr("filtered", "review_star as label")
featurizedData_test = hashingTF.transform(input_df_test)
tfidfModel_test = idf.fit(featurizedData_test).transform(featurizedData_test)

In [501]:
prediction = model.transform(tfidfModel_test)

In [502]:
prediction.where(prediction.prediction != prediction.label).show()

+--------+-----+-----------+--------+-------------+-----------+----------+
|filtered|label|rawFeatures|features|rawPrediction|probability|prediction|
+--------+-----+-----------+--------+-------------+-----------+----------+
+--------+-----+-----------+--------+-------------+-----------+----------+



In [503]:
prediction.show(1)

+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|            filtered|label|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|[enjoyed, delicio...|    5|(262144,[24113,27...|(262144,[24113,27...|[-2.5715539192712...|[8.65336402638732...|       5.0|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 1 row



In [505]:
df_pred = prediction.select(["prediction", "label"]).createTempView("predictions2")

In [506]:
metric_df = spark.sql("SELECT SUM(CASE WHEN prediction = 5 AND label = 5 THEN 1 ELSE 0 END) AS tp, \
                              SUM(CASE WHEN prediction = 1 AND label = 1 THEN 1 ELSE 0 END) AS tn, \
                              SUM(CASE WHEN prediction = 5 AND label = 1 THEN 1 ELSE 0 END) AS fp, \
                              SUM(CASE WHEN prediction = 1 AND label = 5 THEN 1 ELSE 0 END) AS fn \
                      FROM predictions2")

In [507]:
metric_df.show()

+----+----+---+---+
|  tp|  tn| fp| fn|
+----+----+---+---+
|3930|1070|  0|  0|
+----+----+---+---+



In [508]:
matrix, recall, precision, accuracy = cleaner.metrics(df=metric_df)

In [509]:
#confusion matrix
matrix

array([[3930,    0],
       [   0, 1070]])

In [510]:
print("Recall: {}%".format(round(recall*100, 2)))
print("Precision: {}%".format(round(precision*100, 2)))
print("Accuracy: {}%".format(round(accuracy*100, 2)))

Recall: 100.0%
Precision: 100.0%
Accuracy: 100.0%


In [514]:
beta = model.coefficientMatrix.values

In [515]:
len(beta)

90906

### CountVectorizer

In [516]:
cv = CountVectorizer(inputCol="filtered", outputCol="features", minDF=2.0)
model_cv = cv.fit(input_df)
result = model_cv.transform(input_df)

In [517]:
len(model_cv.vocabulary)

8546

In [518]:
lr_model_cv = lr.fit(result)

In [568]:
input_df_test.show(1)

+--------------------+-----+
|            filtered|label|
+--------------------+-----+
|[enjoyed, delicio...|    5|
+--------------------+-----+
only showing top 1 row



In [569]:
#dataframe without labels
input_df_test_cv = input_df_test.select("filtered")

In [576]:
model_cv = cv.fit(input_df_test)
result_test = model_cv.transform(input_df_test)
prediction = lr_model_cv.transform(result_test)

In [577]:
prediction.where(prediction.prediction != prediction.label).show()

+--------+-----+--------+-------------+-----------+----------+
|filtered|label|features|rawPrediction|probability|prediction|
+--------+-----+--------+-------------+-----------+----------+
+--------+-----+--------+-------------+-----------+----------+



In [572]:
len(model_cv.vocabulary)

8546

In [578]:
# prediction = prediction.join(input_df_test, ["filtered"], "label")

In [579]:
df_pred = prediction.select(["prediction", "label"]).createTempView("predictions4")

In [580]:
metric_df = spark.sql("SELECT SUM(CASE WHEN prediction = 5 AND label = 5 THEN 1 ELSE 0 END) AS tp, \
                              SUM(CASE WHEN prediction = 1 AND label = 1 THEN 1 ELSE 0 END) AS tn, \
                              SUM(CASE WHEN prediction = 5 AND label = 1 THEN 1 ELSE 0 END) AS fp, \
                              SUM(CASE WHEN prediction = 1 AND label = 5 THEN 1 ELSE 0 END) AS fn \
                      FROM predictions4")

In [581]:
metric_df.show()

+----+----+---+---+
|  tp|  tn| fp| fn|
+----+----+---+---+
|3930|1070|  0|  0|
+----+----+---+---+



In [527]:
matrix, recall, precision, accuracy = cleaner.metrics(df=metric_df)

In [528]:
#confusion matrix
matrix

array([[3930,    0],
       [   0, 1070]])

In [616]:
lr_model_cv.coefficientMatrix.toSparse


<bound method DenseMatrix.toSparse of DenseMatrix(6, 8546, [-0.0014, -0.0013, -0.0015, -0.0012, -0.002, -0.0009, -0.001, -0.0014, ..., 0.1127, -0.0709, -0.0482, 0.1296, 0.2178, -0.6063, -0.0029, 0.1646], 1)>

In [617]:
lr_model_cv.coefficientMatrix

DenseMatrix(6, 8546, [-0.0014, -0.0013, -0.0015, -0.0012, -0.002, -0.0009, -0.001, -0.0014, ..., 0.1127, -0.0709, -0.0482, 0.1296, 0.2178, -0.6063, -0.0029, 0.1646], 1)

In [530]:
print("Recall: {}%".format(round(recall*100, 2)))
print("Precision: {}%".format(round(precision*100, 2)))
print("Accuracy: {}%".format(round(accuracy*100, 2)))

Recall: 100.0%
Precision: 100.0%
Accuracy: 100.0%


In [534]:
model_cv.vocabulary

['food',
 'great',
 'place',
 'good',
 'service',
 'one',
 'like',
 'madison',
 'time',
 'back',
 've',
 'go',
 'get',
 'best',
 'also',
 'restaurant',
 'really',
 'delicious',
 'ordered',
 'menu',
 'well',
 'us',
 'love',
 'order',
 'cheese',
 'got',
 'pizza',
 'always',
 'even',
 'amazing',
 'friendly',
 'staff',
 'nice',
 'm',
 'chicken',
 'never',
 'try',
 'came',
 'bar',
 'ever',
 'definitely',
 'first',
 'made',
 'dinner',
 'went',
 'wait',
 'sauce',
 'eat',
 'come',
 'people',
 'night',
 'favorite',
 'excellent',
 'little',
 'fresh',
 'make',
 'much',
 're',
 'lunch',
 'table',
 'beer',
 'didn',
 'everything',
 'two',
 'meal',
 'know',
 'experience',
 'recommend',
 'every',
 'minutes',
 'salad',
 'going',
 'way',
 'atmosphere',
 '5',
 'better',
 'coffee',
 'drinks',
 'times',
 'new',
 'sandwich',
 'burger',
 'perfect',
 'tried',
 'right',
 'want',
 'side',
 '2',
 'small',
 'sure',
 'pretty',
 'many',
 'take',
 'awesome',
 'say',
 'day',
 'breakfast',
 'think',
 'area',
 'said',
